In [4]:
%%shell
wget http://deepyeti.ucsd.edu/jmcauley/datasets/librarything/lthing_data.tar.gz

--2022-07-01 10:24:53--  http://deepyeti.ucsd.edu/jmcauley/datasets/librarything/lthing_data.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622688699 (594M) [application/octet-stream]
Saving to: ‘lthing_data.tar.gz’

lthing_data.tar.gz  100%[===================>] 593.84M   103MB/s    in 5.9s    

2022-07-01 10:24:59 (101 MB/s) - ‘lthing_data.tar.gz’ saved [622688699/622688699]



In [6]:
import tarfile
with tarfile.open("lthing_data.tar.gz") as tar:
  print("File in tar archive:")
  tar.list()
  print("Done")

  with tar.extractfile("lthing_data/reviews.json") as file:
    count = 0
    for line in file:
      print(line)
      count += 1
      if count > 3:
        break
  print("Done")

File in tar archive:
?rwxr-xr-x julian/julian          0 2016-09-30 21:58:55 lthing_data/ 
?rw-r--r-- julian/julian    4824989 2014-01-02 18:55:12 lthing_data/edges.txt 
?rw-rw-r-- julian/julian 1604368260 2016-09-30 21:58:25 lthing_data/reviews.json 
Done
b"{'work': '3206242', 'flags': [], 'unixtime': 1194393600, 'stars': 5.0, 'nhelpful': 0, 'time': 'Nov 7, 2007', 'comment': 'This a great book for young readers to be introduced to the world of Middle Earth. ', 'user': 'van_stef'}\n"
b"{'work': '12198649', 'flags': [], 'unixtime': 1333756800, 'stars': 5.0, 'nhelpful': 0, 'time': 'Apr 7, 2012', 'comment': 'Help Wanted: Tales of On The Job Terror from Evil Jester Press is a fun and scary read. This book is edited by Peter Giglio and has short stories by Joe McKinney, Gary Brandner, Henry Snider and many more. As if work wasnt already scary enough, this book gives you more reasons to be scared. Help Wanted is an excellent anthology that includes some great stories by some master storytell

In [7]:
import ast
reviews = []
with tarfile.open("lthing_data.tar.gz") as tar:
  with tar.extractfile("lthing_data/reviews.json") as file:
    for line in file:
      record = ast.literal_eval(line.decode("utf8"))
      if any( x not in record for x in ['user', 'work', 'stars']):
        continue
      reviews.append([record['user'], record['work'], record['stars']])
print(len(reviews), "records retrieved")


1387209 records retrieved


In [10]:
import pandas as pd
reviews = pd.DataFrame(reviews, columns=['user', 'work', 'stars'])
print(reviewse)

                  user      work  stars
0             van_stef   3206242    5.0
1             dwatson2  12198649    5.0
2             amdrane2  12981302    4.0
3        Lila_Gustavus   5231009    3.0
4            skinglist    184318    2.0
...                ...       ...    ...
1387204        wirtley  10051104    5.0
1387205         drruth     94456    5.0
1387206       jclark88   8253945    3.0
1387207         Jellyn   5377722    3.0
1387208        koeniel    452711    4.0

[1387209 rows x 3 columns]


In [11]:
# looking for users who reviewed more than 50 books
usercount = reviews[['work', 'user']].groupby('user').count()
usercount = usercount[usercount['work'] >= 50]
print(usercount)

            work
user            
              84
-Eva-        602
06nwingert   370
1983mk        63
1dragones    194
...          ...
zjakkelien   121
zmagic69     148
zquilts       67
zwaantje     121
zzshupinga   246

[5736 rows x 1 columns]


In [14]:
# Look for the books that were reviewed by more than 50 users
workcount = reviews[["work","user"]].groupby("work").count()
workcount = workcount[workcount["user"] >= 50]
print(workcount)

          user
work          
10000       74
1000167    106
10005525   107
10007394    61
10007399    56
...        ...
9964442     53
9978        58
9979582    193
9989655     69
9993        51

[1595 rows x 1 columns]


In [13]:
...
# Keep only the popular books and active users
reviews = reviews[reviews["user"].isin(usercount.index) & reviews["work"].isin(workcount.index)]
print(reviews)

                user     work  stars
0           van_stef  3206242    5.0
6            justine     3067    4.5
18           stephmo  1594925    4.0
19         Eyejaybee  2849559    5.0
35       LisaMaria_C   452949    4.5
...              ...      ...    ...
1387161     connie53     1653    4.0
1387177   BruderBane    24623    4.5
1387192  StuartAston  8282225    4.0
1387202      danielx  9759186    4.0
1387206     jclark88  8253945    3.0

[205110 rows x 3 columns]


In [17]:
reviewmatrix = reviews.pivot(index="user", columns="work", values="stars").fillna(0)
print(reviewmatrix)
print(reviewmatrix.shape)

work        10000  10001  1000167  10001797  10005525  10007394  10007399  \
user                                                                        
              0.0    0.0      0.0       0.0       0.0       0.0       0.0   
-Eva-         0.0    0.0      0.0       0.0       0.0       0.0       0.0   
06nwingert    0.0    0.0      0.0       0.0       0.0       0.0       0.0   
1983mk        0.0    0.0      0.0       0.0       0.0       0.0       0.0   
1dragones     5.0    4.0      0.0       0.0       0.0       0.0       0.0   
...           ...    ...      ...       ...       ...       ...       ...   
zjakkelien    0.0    0.0      0.0       0.0       0.0       0.0       0.0   
zmagic69      0.0    0.0      0.0       0.0       0.0       0.0       0.0   
zquilts       0.0    0.0      0.0       0.0       0.0       0.0       0.0   
zwaantje      0.0    0.0      0.0       0.0       0.0       0.0       0.0   
zzshupinga    0.0    0.0      0.0       0.0       0.0       0.0       0.0   

In [19]:
from numpy.linalg import svd
matrix = reviewmatrix.values
u, s, vh = svd(matrix, full_matrices=False)

In [22]:
import numpy as np
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))
 
highest_similarity = -np.inf
highest_sim_col = -1
for col in range(1,vh.shape[1]):
    similarity = cosine_similarity(vh[:,0], vh[:,col])
    if similarity > highest_similarity:
        highest_similarity = similarity
        highest_sim_col = col
 
print("Column %d is most similar to column 0" % highest_sim_col)

Column 598 is most similar to column 0
